In [468]:
# widely used imports
import pandas as pd
import numpy as np
import os
import numpy.random as random
import time
import functools
import datetime

# working directory
os.chdir("/Users/burke/Documents/research/scrooge")

# import key NHATS fields
nhanesDF = pd.read_stata("nhanesForScrooge.dta")

# simple weighting schema using the NHATS weights and turnign into simple probability weights
nhanesDF['probWeight'] = nhanesDF.WTINT2YR / np.sum(nhanesDF.WTINT2YR) 
nhanesDF.patientID.astype("int64")

# load the file mapping screening services to patients
screeningRules = pd.read_excel("simplifiedPreventiveServices.xlsx")
screeningRules['screeningIndex'] = np.arange(0, len(screeningRules))

In [175]:
class Person:
    def __init__(self, patientID, gender, age, race, bmi, dm, htn, hl, smoking):
        self.patientID = patientID
        self.gender = gender
        self.age = age
        self.bmi = bmi
        self.dm = dm
        self.htn = htn
        self.hl = hl
        self.smoking = smoking

In [469]:
class ScreeningElement:
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):
        return true

class AgeScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):
        if (screeningRule['minAge'] != None and visit.age < screeningRule['minAge']):
            return False
        if (screeningRule['maxAge'] != None and visit.age > screeningRule['maxAge']):
            return False
        return True

class GenderScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        if (screeningRule['Gender'] != None and visit.gender != screeningRule['Gender']):
            return False
        return True

class SmokingScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        if (screeningRule['Smoking'] != None):
            if (screeningRule['Smoking'] == 'Current' and visit.smokingStatus != 1):
                return False
            if (screeningRule['Smoking'] == 'Former' and visit.smokingStatus == 0):
                return False
            if (screeningRule['Smoking']  == 'Never' and visit.smokingStatus != 0):
                return False
        return True

class VascularRiskFactorScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        if (screeningRule['VascularRiskFactor'] != None):
            if (screeningRule['VascularRiskFactor'] == 1 and (visit.selfReportHtn == 'No' or visit.selfReportHtn is None) 
                and (visit.selfReportHyperlipidemia == 'No' or visit.selfReportHyperlipidemia is None)
                and visit.selfReportDiabetes == 'No' or visit.selfReportDiabetes is None):
                return False
            if (screeningRule['VascularRiskFactor'] == 0 and (visit.selfReportHtn == 'Yes' or visit.selfReportHyperlipidemia == 'Yes' or visit.selfReportDiabetes == 'Yes')):
                return False
        return True
    
class BMIScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        if ((screeningRule['bmiOver'] != None and visit.bmi <= screeningRule['bmiOver']) or 
           (screeningRule['bmiOver'] != None and visit.bmi is None)):
            return False
        return True

class RiskProbabilityScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        # this will set the same random seed for each patietn/screening rule cobination. so, a patient will have an 
        # independt "risk" for each separate rule, but when a separate visit is evaluated it'll be teh same risk as for the 
        # last visit
        random.seed(visit.patientID * screeningRule.screeningIndex)
        if (screeningRule['proportionOfPopulationAtRisk'] != None and random.random() > (screeningRule['proportionOfPopulationAtRisk'])):
            return False
        return True

class TimingScreeningElement(ScreeningElement):
    def elementAppliesToRule(self, screeningRule, visit, priorScreenings):    
        # next step is to check to see if the rule has been implemented since the last rule interval..        
        hasPriorScreeningsForService = (priorScreenings['Service'] == screeningRule['Service']).any()
        # if the screening rule is a one time rule and its ever been implemented, then don't do it again


        # ugly — there is really complicated branching logic here that i dont' like...it does ok on tests, so i'm 
        # not terrifed of it...but this is a good place to think about refactoring.
        if (screeningRule['Frequency'] == "Once" and hasPriorScreeningsForService):
            return False
        # if the screening rule is repeated...check whether its been repeated since a prior visit
        elif (hasPriorScreeningsForService):  
            priorScreeningsForThisService = priorScreenings.loc[(priorScreenings['Service'] == screeningRule['Service']) & (priorScreenings['applicable'] == True)].sort_values("visitDate", ascending=False)
            if (datetime.datetime.strptime(visit.visitDate, "%m/%d/%Y") - 
                  datetime.timedelta(days=(365*int(screeningRule['Frequency'][1]))) < 
                  datetime.datetime.strptime(priorScreeningsForThisService.iloc[0]['visitDate'], "%m/%d/%Y")):
                return False
            else:
                return True
        else:
            return True

In [548]:
parameter_annualPanelAttritionRate = 0.30
parameter_proportionOfAllVisitsToPCP = 0.51

# tuple indexed dictionaries. first tuple element is the lower bounds of an age group and second element is the upper bounds
# the mtached value is the # of visits per 100 members of the population within a given age/gender band
parameter_maleVisitRates = {(18,24) : 119.6, (25,44) : 127.3, (45,64) : 312.1, (65,74) : 559.5, (75,80): 799.2}
parameter_femaleVisitRates = {(18,24) : 235.3, (25,44) : 302.7, (45,64) : 417.3, (65,74) : 606.7, (75,80): 736.2}

visitColumns = ['visitDate', 'patientID', 'year', 'age', 'gender', 'raceEth', 'bmi',
                'smokingStatus', 'selfReportHtn', 'selfReportHyperlipidemia', 'selfReportDiabetes']

screeningColumns = ['patientID', 'Service', 'applicable', 'timeSpent', 'visitDate']

# the major design decision is whether to build python objects for each of the conceptual steps —
# patient, visit, screening service...or whether to keep them all as data frames at the provider level.
# there isn't going to be a ton of logic at any of those levels, so its feasiable to just have a provider level 
# object. from a performace perspective, i'm sure that operations on dataframes are going to be better on memory
# and i suspect they'll probably also be a lot quicker.

class Provider:
    def __init__(self, panelSize):
        self.panelSize = panelSize
        self.initPanel()
        self.visits = pd.DataFrame(data=None, columns=visitColumns)
        self.visits.patientID.astype("int")
    
        self.screeningServices = pd.DataFrame(data=None, columns=screeningColumns)
        self.startYear = 2018
        self.year = self.startYear
        # this dict will keep a separate set of dataframes for the screening services for a given patient
        self.screeningsForPatientID = {}
        self.screeningElements = [AgeScreeningElement(), GenderScreeningElement(), VascularRiskFactorScreeningElement(),
                                  SmokingScreeningElement(), BMIScreeningElement(), RiskProbabilityScreeningElement(), TimingScreeningElement()]                                
        
    def initPanel(self):
        rowIndices = random.choice(nhanesDF.index.values, size = self.panelSize, replace=True, p=nhanesDF.probWeight)
        self.panel = nhanesDF.iloc[rowIndices]
        self.panel = self.panel.reset_index(drop=True)
        # not sure that we'll need this...but, lifetime panel is going to keep track of every patient that was ever part of a panel — including those that fall out
        self.lifetimePanel = self.panel.copy()
    
    def addScreeningForPatient(self, patientID, screening):
        if patientID in self.screeningsForPatientID:
            self.screeningsForPatientID[patientID] = self.screeningsForPatientID[patientID].append(screening, ignore_index=True)
        else:
            self.screeningsForPatientID[patientID] = pd.DataFrame(data=screening, index=[0])

    def getScreeningsForPatient(self, patientID):
        if patientID in self.screeningsForPatientID:
            return self.screeningsForPatientID[patientID]
        else:
            return pd.DataFrame(data=None, columns=screeningColumns)
    
    def advancePanelByYear(self, years):
        for i in range(0, years):
            self.year += 1
            self.losePatientsToAttrition(parameter_annualPanelAttritionRate)
            self.addNewPatients(parameter_annualPanelAttritionRate)
            self.generateVisitHistoryForPanel()
        
    def losePatientsToAttrition(self, attritionRate):
        self.panel = self.panel.drop(random.choice(self.panel.index.values, size=int(attritionRate * self.panelSize), replace=False)) 
    
    def addNewPatients(self, attritionRate):
        newRowIndices = random.choice(nhanesDF.index.values, size = int(attritionRate * self.panelSize), replace=True, p=nhanesDF.probWeight)
        self.panel = self.panel.append(nhanesDF.iloc[newRowIndices])
        self.lifetimePanel = self.lifetimePanel.append(nhanesDF.iloc[newRowIndices])
        self.panel = self.panel.reset_index(drop=True)
    
    def generateVisitHistoryForPanel(self):
        men = self.panel.loc[self.panel['gender'] == 'Male']
        women = self.panel.loc[self.panel['gender'] == 'Female']
        
        self.generateVisitsForGender(men, parameter_maleVisitRates)
        self.generateVisitsForGender(women, parameter_femaleVisitRates)
        self.applyScreeningsToVisits()              

    
    def applyScreeningRulesToVisit(self, visit):
        data = []
        for blank, screeningRule in screeningRules.iterrows():
            data.append(self.applyScreeningRuleToVisit(visit, screeningRule))
        return data
    
    def applyScreeningsToVisits(self):
        data = []
        for blank, visit in self.visits.iterrows():
            data.extend(self.applyScreeningRulesToVisit(visit))
        screeningDF = pd.DataFrame(data) 
        self.screeningServices = pd.concat([self.screeningServices, screeningDF])  
        
    def applyScreeningRuleToVisit(self, visit, screeningRule):
        applies = True
        priorScreenings = self.getScreeningsForPatient(visit.patientID)
        
        firstElementToFail = None
        for element in self.screeningElements:
            applies = element.elementAppliesToRule(screeningRule, visit, priorScreenings)
            # for troubleshooting
            if (applies is False and firstElementToFail is None):
                firstElementToFail = element
            # as soon as one element fails...then you don't have to check the rest
            if (applies is False):
                break            
        '''        
        if (firstElementToFail is not None):
            print "Failed at: " + str(firstElementToFail.__class__.__name__)
        '''
        newScreeningService = {'patientID' : visit.patientID, 'Service' : screeningRule['Service'],
                                'applicable' : applies, 'timeSpent' : screeningRule['Time'] if applies else 0,
                                'visitDate' : visit.visitDate}
        self.addScreeningForPatient(visit.patientID, newScreeningService)
        return newScreeningService
    
    def generateVisitsForGender(self, patients, visitRatesByAge):
        for ageRange in visitRatesByAge.keys():
            patientsWithinAgeRange = patients.loc[(patients['age'] >= ageRange[0]) & (patients['age'] <= ageRange[1])]
            totalVisits = int(visitRatesByAge[ageRange] * len(patientsWithinAgeRange) * parameter_proportionOfAllVisitsToPCP/ 100)
            patients = self.panel.loc[random.choice(self.panel.index.values, size=totalVisits, replace=True)]
            timesForVisits = self.generateDatesForVisits("1/1/" + str(self.year), "12/31/" + str(self.year+1), len(patients))
            newVisits = pd.DataFrame(data={"visitDate" : timesForVisits, "patientID" : patients.patientID.values, 
                                           'year' : [self.year] * len(patients), 'age' : patients.age.values,
                                          'gender' : patients.gender.values, 'raceEth' : patients.raceEth.values, 'bmi' : patients.bmi.values,
                                          'smokingStatus' : patients.smokingStatus.values, 'selfReportHtn' : patients.selfReportHtn.values,
                                          'selfReportHyperlipidemia': patients.selfReportHyperlipidemia.values,
                                           'selfReportDiabetes' : patients.selfReportDiabetes.values})
            self.visits = self.visits.append(newVisits)

    def generateDatesForVisits(self, startTime, endTime, numTimes):
        times = []
        stime = time.mktime(time.strptime(startTime, "%m/%d/%Y"))
        etime = time.mktime(time.strptime(endTime, "%m/%d/%Y"))

        for i in range(0, numTimes):
            times.append(time.strftime("%m/%d/%Y", time.localtime(stime + random.rand() * (etime-stime))))
            
        return times

In [549]:
provider = Provider(panelSize=50)
provider.advancePanelByYear(1)

In [550]:
provider.screeningServices

,Service,applicable,patientID,timeSpent,visitDate
0,Abdominal Aortic Aneurysm: Screening,False,84816,0,06/11/2020
1,Abdominal Aortic Aneurysm: Screening,False,84816,0,06/11/2020
2,Tobacco Smoking Cessation in Adults: Behaviora...,False,84816,0,06/11/2020
3,Syphilis Infection in Nonpregnant Adults and A...,False,84816,0,06/11/2020
4,Statin Use for the Primary Prevention of Cardi...,False,84816,0,06/11/2020
5,Statin Use for the Primary Prevention of Cardi...,False,84816,0,06/11/2020
6,Skin Cancer Prevention: Behavioral Counseling,False,84816,0,06/11/2020
7,Skin Cancer Prevention: Behavioral Counseling,False,84816,0,06/11/2020
8,Sexually Transmitted Infections: Behavioral Co...,False,84816,0,06/11/2020
9,Sexually Transmitted Infections: Behavioral Co...,False,84816,0,06/11/2020


## Unit Tests of the Logic of applying screens to visits

In [490]:
import unittest
ageScreeningRule = pd.Series({"Service" : "AgeFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : 18, "maxAge" : 40, 
                 "Gender" : "Male", "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})

genderScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "Once", "minAge" : None, "maxAge" : None, 
                 "Gender" : "Male", "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})
currentSmokerScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : "Current",
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})
formerSmokerScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : "Former",
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})
neverSmokerScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : "Never",
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})

hasVFScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : 1, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})
hasNoVFScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : 0, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})

bmiScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : 0, "bmiOver" : 30, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})

completeRiskScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : 1.0, "screeningIndex" : 42})

zeroRiskScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "q1", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : 0.0, "screeningIndex" : 42})

universalOnceScreeningRule = pd.Series({"Service" : "GenderFilter", "Grade" : "A", "Frequency" : "Once", "minAge" : None, "maxAge" : None, 
                 "Gender" : None, "VascularRiskFactor" : None, "bmiOver" : None, "Time" : 2, "Smoking" : None,
                 "proportionOfPopulationAtRisk" : None, "screeningIndex" : 42})



visit15YearOldMale = pd.Series({'visitDate' : "1/1/2019", 'patientID' : 123, 'year' : 2015, 'age' : 15, 'gender' : 'Male', 
                      'raceEth' : 1, 'bmi' : None, 'smokingStatus' : 1, 'selfReportHtn' : None ,
                      'selfReportHyperlipidemia' : "No", 'selfReportDiabetes' : "No"})

visit19YearOldMale = visit15YearOldMale.copy()
visit19YearOldMale['age'] = 19 
visit39YearOldMale = visit15YearOldMale.copy()
visit39YearOldMale['age'] = 39
visit41YearOldMale = visit15YearOldMale.copy()
visit41YearOldMale['age'] =  41
visit39YearOldFemale = visit39YearOldMale.copy()
visit39YearOldFemale['gender'] = 'Female'
visit15YearOldMaleFormerSmoker= visit15YearOldMale.copy()
visit15YearOldMaleFormerSmoker['smokingStatus'] = 2 # former
visit15YearOldMaleNeverSmoker= visit15YearOldMale.copy()
visit15YearOldMaleNeverSmoker['smokingStatus'] = 0 # never
visit15YearOldMaleHypertension= visit15YearOldMale.copy()
visit15YearOldMaleHypertension['selfReportHtn'] = "Yes"
visit15YearOldMaleHyperlipidemia= visit15YearOldMale.copy()
visit15YearOldMaleHyperlipidemia['selfReportHyperlipidemia'] = "Yes"
visit15YearOldMaleDiabetes= visit15YearOldMale.copy()
visit15YearOldMaleDiabetes['selfReportDiabetes'] = "Yes"
visit15YearOldMaleAllRiskFactors= visit15YearOldMale.copy()
visit15YearOldMaleAllRiskFactors['selfReportDiabetes'] = "Yes"
visit15YearOldMaleAllRiskFactors['selfReportHyperlipidemia'] = "Yes"
visit15YearOldMaleAllRiskFactors['selfReportHtn'] = "Yes"
visit15YearOldMaleLowBMI = visit15YearOldMale.copy()
visit15YearOldMaleLowBMI['bmi'] = 18
visit15YearOldMaleHighBMI = visit15YearOldMale.copy()
visit15YearOldMaleHighBMI['bmi'] = 35



class TestScreeningRules(unittest.TestCase):   
    def testTimingInterval(self):
        dummyProvider = Provider(0)
        # apply the rule so, that there is a screening visit on 1/1/19
        self.assertTrue(dummyProvider.applyScreeningRuleToVisit(visit19YearOldMale, ageScreeningRule)['applicable'])
        
        visit19YearOldMaleAdvance6Months = visit19YearOldMale.copy()
        visit19YearOldMaleAdvance6Months['visitDate'] = "7/1/2019"
        self.assertFalse( dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance6Months, ageScreeningRule)['applicable'])
        
        visit19YearOldMaleAdvance11Months = visit19YearOldMale.copy()
        visit19YearOldMaleAdvance11Months['visitDate'] = "12/31/2019"
        self.assertFalse( dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance11Months, ageScreeningRule)['applicable'])
        
        visit19YearOldMaleAdvance13Months = visit19YearOldMale.copy()
        visit19YearOldMaleAdvance13Months['visitDate'] = "1/2/2020"

        self.assertTrue( dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance13Months, ageScreeningRule)['applicable'])

        visit19YearOldMaleAdvance24Months = visit19YearOldMale.copy()
        visit19YearOldMaleAdvance24Months['visitDate'] = "1/2/2021"
        self.assertTrue( dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance24Months, ageScreeningRule)['applicable'])
    
    def testTimingOnce(self):
        dummyProvider = Provider(0)
        # apply the rule the first time, and it shoudl go through...
        self.assertTrue(dummyProvider.applyScreeningRuleToVisit(visit19YearOldMale, universalOnceScreeningRule)['applicable'])
        # apply it subsequent and it should fail
        self.assertFalse(dummyProvider.applyScreeningRuleToVisit(visit19YearOldMale, universalOnceScreeningRule)['applicable'])

    # hard to do this test determinisstically, will just test the extreme probabilities
    def testRiskScreening(self):
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit19YearOldMale, completeRiskScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleAllRiskFactors, completeRiskScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleAllRiskFactors, zeroRiskScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleAllRiskFactors, zeroRiskScreeningRule)['applicable'])
    
    def testBMI(self):
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, bmiScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleLowBMI, bmiScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleHighBMI, bmiScreeningRule)['applicable'])
    
    def testVascularRiskFactorFilter(self):
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, hasNoVFScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, hasVFScreeningRule)['applicable'])

        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleHypertension, hasNoVFScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleHypertension, hasVFScreeningRule)['applicable'])

        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleHyperlipidemia, hasNoVFScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleHyperlipidemia, hasVFScreeningRule)['applicable'])

        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleDiabetes, hasNoVFScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleDiabetes, hasVFScreeningRule)['applicable'])

        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleAllRiskFactors, hasNoVFScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleAllRiskFactors, hasVFScreeningRule)['applicable'])
    
    def testSmokingFilter(self):
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, currentSmokerScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, formerSmokerScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, neverSmokerScreeningRule)['applicable'])

        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleFormerSmoker, currentSmokerScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleFormerSmoker, formerSmokerScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleFormerSmoker, neverSmokerScreeningRule)['applicable'])
        
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleNeverSmoker, currentSmokerScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleNeverSmoker, formerSmokerScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMaleNeverSmoker, neverSmokerScreeningRule)['applicable'])
    
    def testGenderFilter(self):
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, genderScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit19YearOldMale, genderScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit39YearOldMale, genderScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit39YearOldMale, genderScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit39YearOldFemale, genderScreeningRule)['applicable'])
    
    def testAgeFilter(self):
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit15YearOldMale, ageScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit19YearOldMale, ageScreeningRule)['applicable'])
        self.assertTrue(Provider(0).applyScreeningRuleToVisit(visit39YearOldMale, ageScreeningRule)['applicable'])
        self.assertFalse(Provider(0).applyScreeningRuleToVisit(visit41YearOldMale, ageScreeningRule)['applicable'])
    
   
suite = unittest.TestLoader().loadTestsFromTestCase(TestScreeningRules)
unittest.TextTestRunner(verbosity=2).run(suite)      

testAgeFilter (__main__.TestScreeningRules) ... ok
testBMI (__main__.TestScreeningRules) ... ok
testGenderFilter (__main__.TestScreeningRules) ... ok
testRiskScreening (__main__.TestScreeningRules) ... ok
testSmokingFilter (__main__.TestScreeningRules) ... ok
testTimingInterval (__main__.TestScreeningRules) ... ok
testTimingOnce (__main__.TestScreeningRules) ... ok
testVascularRiskFactorFilter (__main__.TestScreeningRules) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.275s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

In [459]:
dummyProvider = Provider(0)
# apply the rule so, that there is a screening visit on 1/1/19
dummyProvider.applyScreeningRuleToVisit(visit19YearOldMale, ageScreeningRule)
visit19YearOldMaleAdvance13Months = visit19YearOldMale.copy()
visit19YearOldMaleAdvance13Months['visitDate'] = "1/2/2020"
dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance13Months, ageScreeningRule)
visit19YearOldMaleAdvance15Months = visit19YearOldMale.copy()
visit19YearOldMaleAdvance15Months['visitDate'] = "4/2/2020"
dummyProvider.applyScreeningRuleToVisit(visit19YearOldMaleAdvance15Months, ageScreeningRule)

has prior screnings: False
has prior screnings: True
     Service  applicable  patientID  timeSpent visitDate
0  AgeFilter        True        123          2  1/1/2019
prior date: 2019-01-01 00:00:00
visit date: 2020-01-02 00:00:00
subtracted date date: 2019-01-02 00:00:00
has prior screnings: True
     Service  applicable  patientID  timeSpent visitDate
1  AgeFilter        True        123          2  1/2/2020
0  AgeFilter        True        123          2  1/1/2019
prior date: 2020-01-02 00:00:00
visit date: 2020-04-02 00:00:00
subtracted date date: 2019-04-03 00:00:00
Failed at: TimingScreeningElement


{'Service': 'AgeFilter',
 'applicable': False,
 'patientID': 123,
 'timeSpent': 0,
 'visitDate': '4/2/2020'}